### External downloads

Model weights available for download as .pt file on Google Drive.

### Model config

In [ ]:
CFG_PATH = "../Towards-Realtime-MOT/cfg/yolov3_1088x608.cfg"
WEIGHTS_PATH = "../../../../jihon-blob/jde.1088x608.uncertainty.pt"
IOU_THRESHOLD = 0.5
CONF_THRESHOLD = 0.5
NMS_THRESHOLD = 0.4
MIN_BOX_AREA = 200
TRACKING_BUFFER = 30

### Data config

In [ ]:
INPUT_VIDEO_PATH = "../../../data/benfica/sample.mp4"
OUTPUT_FORMAT = "video"
OUTPUT_ROOT_PATH = "../../../data/benfica/trmot-tracking-output"

### Setup

In [ ]:
import sys
sys.path.insert(1, "../Towards-Realtime-MOT")

In [ ]:
import logging
import argparse
from utils.utils import *
from utils.log import logger
from utils.timer import Timer
from utils.parse_config import parse_model_cfg
import utils.datasets as datasets
from track import eval_seq

In [ ]:
logger.setLevel(logging.INFO)

### Run

In [ ]:
## We either need to bundle the parameters back into OPT (kinda weird because this is not run from command line)
## or refactor eval_seq (in top-level track.py) ourselves
result_root = opt.output_root if opt.output_root != "" else "."
mkdir_if_missing(result_root)

# cfg_dict = parse_model_cfg(opt.cfg)
cfg_dict = parse_model_cfg(CFG_PATH="../Towards-Realtime-MOT/cfg/yolov3_1088x608.cfg")
opt.img_size = [int(cfg_dict[0]["width"]), int(cfg_dict[0]["height"])]

# run tracking
timer = Timer()
accs = []
n_frame = 0

logger.info("Starting tracking...")
dataloader = datasets.LoadVideo(opt.input_video, opt.img_size)
result_filename = os.path.join(result_root, "results.txt")
frame_rate = dataloader.frame_rate

frame_dir = None if opt.output_format == "text" else osp.join(result_root, "frame")
try:
    eval_seq(
        opt,
        dataloader,
        "mot",
        result_filename,
        save_dir=frame_dir,
        show_image=False,
        frame_rate=frame_rate,
    )
except Exception as e:
    logger.info(e)

if opt.output_format == "video":
    output_video_path = osp.join(result_root, "result.mp4")
    cmd_str = "ffmpeg -f image2 -i {}/%05d.jpg -c:v copy {}".format(
        osp.join(result_root, "frame"), output_video_path
    )
    os.system(cmd_str)